# Part2 深度学习实践
1-4章主要是介绍深度学习，以及其工作原理，5-9章将通过实践培养出如何用深度学习解决实际问题。
# 第5章 深度学习用于计算机视觉
理解卷积神经网络  
使用数据增强来降低过拟合  
使用预训练的卷积神经网络进行特征提取  
微调预训练的卷积神经网络  
将卷积神经网络学到的内容及其如何做出分类决策可视化
## 5.4 卷积神经网络简介
深入介绍神经网络原理，以及在计算机视觉任务上为何如此成功。下面是Conv2D层和MaxPooling2D层的堆叠。

In [ ]:
#